In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import seaborn as sns
import zipfile
import re

In [ ]:
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["axes.titlesize"] = 18

# Loading dataframes

In [ ]:
VOTE_DATA_PATH = 'data/votes/'
LOBBY_DATA_PATH = 'data/lobby/'

Z_FILE_VOTE = zipfile.ZipFile(VOTE_DATA_PATH+'votes.zip')

vote_data_files = [x.filename for x in Z_FILE_VOTE.infolist() if x.filename.startswith('votes_')]

In [ ]:
def interpret_position(positions):
    r = positions.copy()
    r = r.apply(lambda x : 1 if x == "Yes" else ( -1 if x == "No" else 0 ))
    
    return r

In [ ]:
USEFUL_COLUMNS = ['id','bill.number','congress','member_id','question','position', 'timestamp']

def process_vote_file(file_path):
    print("Processing vote data for {file}".format(file=file_path))
    ifile = Z_FILE_VOTE.open(file_path)
    
    # Read data
    df = pd.read_csv(ifile)

    # We drop entries without bill number as they lead to invalid bill IDs
    df = df[df['bill.number'].notna()]
    
    # We drop entries without roll call value
    df = df[df['roll_call'].notna()]
    
    # As there can be several votes per bill we need to build a unique ID
    df['id'] = df['bill.bill_id'].map(str) + "-" + df['roll_call'].map(str)

    # Some bill IDs and roll call numbers are poorly filled, leading to duplicates
    df = df.drop_duplicates('id')
    
    # Build timestamps
    df['timestamp'] = pd.to_datetime(df.date.map(str) + " " + df.time.map(str))
    
    # Keep only useful columns
    df = df[USEFUL_COLUMNS]
    
    # Convert position to numeric
    df['position'] = interpret_position(df['position'])
    
    return df

In [ ]:
NOMINATION_VOTE = "On the Nomination"
MOTION_VOTE = "On the Motion"
CLOTURE_MOTION_VOTE = "On the Cloture Motion"
BILL_PASSAGE = "On Passage of the Bill"

def build_vote_matrix(questions, data):
    """
    Given a list of questions, build a vote matrix.
    Index are senators IDs, columns are vote IDs and values are the respective positions
    of the senators with respect to the question (Yes, No, blank or NaN if the senator
    did not take part in the vote)
    
    Data should contain the following columns:
        - senators ID (member_id)
        - vote ID (id)
        - position (position)
    """
    vote_matrix = data[data['question'].isin(questions)]
    vote_matrix = vote_matrix.pivot(columns='id', index='member_id', values='position')
    
    return vote_matrix

In [ ]:
# Example vote matrix
vm_data1 = process_vote_file(vote_data_files[210])
vm_data2 = process_vote_file(vote_data_files[202])
vm_data = pd.concat([vm_data1.head(10), vm_data2.head(10)]) 

display(vm_data)

vm = build_vote_matrix([NOMINATION_VOTE,CLOTURE_MOTION_VOTE,MOTION_VOTE], vm_data)

display(vm)

In [ ]:
# Compiling senators from all congresses
raw_senators = pd.DataFrame()

for i in range (115, 80 - 1, -1):
    df = pd.read_csv("data/senate_members/senate_members_{congress}.csv".format(congress = i))
    df['congress'] = i
    raw_senators = pd.concat([raw_senators, df], sort=False)
    
    
# Compiling commities
raw_commities = pd.DataFrame()

for i in range (115, 110 - 1, -1):
    df = pd.read_csv("data/commities/commities_{congress}.csv".format(congress = i))
    df['congress'] = i
    raw_commities = pd.concat([raw_senators, df], sort=False)
    
    
# Retrieving votes
raw_votes = pd.read_csv(Z_FILE_VOTE.open("votes.csv"))

#Retrieving lobbies
raw_lobbies = pd.read_csv(zipfile.ZipFile(LOBBY_DATA_PATH+'lobby.zip').open('lobby.csv'))

In [ ]:
# Example vote matrix
vm_data1 = process_vote_file(vote_data_files[200])
vm_data2 = process_vote_file(vote_data_files[202])
vm_data = pd.concat([vm_data1.head(10), vm_data2.head(10)]) 

display(vm_data)

vm = build_vote_matrix([NOMINATION_VOTE, BILL_PASSAGE, MOTION_VOTE], vm_data)

display(vm)

In [ ]:
df_list = []

for filepath in vote_data_files:
    vm_data = process_vote_file(filepath)
    df_list.append(vm_data)
    
vm_data = pd.concat(df_list)

In [ ]:
vm_data_115 = vm_data[vm_data['congress'] == 115]

vm = build_vote_matrix([BILL_PASSAGE], vm_data_115).dropna(axis=0, how='any')

display(vm)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
pca.fit(vm)

coords = pca.transform(vm)

In [ ]:
raw_senators.columns

In [ ]:
dems = raw_senators[raw_senators['party']=='D']['id']
reps = raw_senators[raw_senators['party']=='R']['id']

In [ ]:
vm_dems = vm[vm.index.isin(dems.values)]
vm_reps = vm[vm.index.isin(reps.values)]

In [ ]:
dems_coords = pca.transform(vm_dems)
reps_coords = pca.transform(vm_reps)

In [ ]:
random_comp = np.random.normal(0,1, len(dems_coords))
plt.scatter(dems_coords, random_comp, color='b')
random_comp = np.random.normal(0,1, len(reps_coords))
plt.scatter(reps_coords, random_comp, color='r')
plt.ylim(-10,10)
plt.show()

In [ ]:
raw_senators.columns

In [ ]:
# Do senators tend to vote more and more with their party ?

data = raw_senators[raw_senators['congress'] > 100]
ax = data.boxplot(column='votes_with_party_pct', by='congress', figsize=(15,10))
ax.set(xlabel="Congress", ylabel="Percentage of votes with party")
ax.get_figure().suptitle("")

plt.title("Evolution of voting with party percentage")
plt.show()

In [ ]:
gender_count = raw_senators[['id', 'gender', 'congress']].groupby(['congress', 'gender']).count()
gc = gender_count.unstack()
gc.columns = gc.columns.droplevel()
# Compute gender percentages
gc = gc.div(gc.sum(axis=1),axis=0).fillna(0)
gc['F'].plot(kind='line', figsize=(15,10), marker='o', linestyle='-')
plt.title("Proportion of women in senate")
plt.show()

In [ ]:
raw_votes.columns

In [ ]:
raw_votes.head()

In [ ]:
dems

### Lobby

In [ ]:
df_lobby = raw_lobbies[['lobbyists','id','lobbying_client.general_description','lobbying_client.name','lobbying_registrant.general_description','lobbying_registrant.name']].copy()

In [ ]:
df_senators = raw_senators.copy()
df_senators['Name'] = df_senators['first_name'] + ' ' +  df_senators['last_name']

In [ ]:
def get_names(x):
    m = re.findall("'name': '(?P<name>[^']+)'",x)
    return [a.title() for a in m]

df_lobby['name'] = df_lobby['lobbyists'].apply(lambda x: get_names(x))
df_lobby_expend = df_lobby.set_index('id')['name'].apply(pd.Series).stack().to_frame().reset_index()
df_lobby_expend = df_lobby_expend.reset_index()
df_lobby_expend = df_lobby_expend.rename(columns = {0:'Name'})
df_lobby_expend = pd.merge(df_lobby_expend,df_lobby,on='id')
df_lobby_expend = df_lobby_expend.drop('lobbyists',1)

In [ ]:
df_merge_lobby = pd.merge(df_lobby_expend,df_senators,on='Name')
display(df_merge_lobby.head(3))
print('Number of unique senators in lobbies: {}'.format(df_merge_lobby['Name'].unique().size))

In [ ]:
def draw_lobby(df,number):
    df_temp = df.groupby('lobbying_client.name')['Name'].nunique().reset_index().sort_values('Name',ascending=False)
    
    plt.figure(figsize=(15,5))
    plt.plot(range(df_temp['Name'].shape[0]), df_temp['Name'])
    plt.xlabel('Lobby client')
    plt.ylabel('Number of Senator')
    plt.ylim(0,4)
    plt.title('Number of distinct senators per lobby client')

    plt.figure(figsize=(15,5))
    df_temp[['Name']].plot.kde()
    plt.xlabel('Number of Senator')
    plt.title('Density of distinct senators per lobby client')
    
    df_temp = df_temp.iloc[:number]
    
    plt.figure(figsize=(15,5))
    plt.bar(range(df_temp.shape[0]), df_temp['Name'], tick_label=df_temp['lobbying_client.name'])
    plt.xticks(rotation=90)
    plt.title('First {} companies with the most distinct senators'.format(number))
    plt.ylabel('Number of Senator')
    plt.show()
    
    df_temp = df.groupby('lobbying_client.name')['Name'].count().reset_index().sort_values('Name',ascending=False).iloc[:number]

    plt.figure(figsize=(15,5))
    plt.bar(range(df_temp.shape[0]), df_temp['Name'], tick_label=df_temp['lobbying_client.name'])
    plt.xticks(rotation=90)
    plt.title('First {} companies with the most senators'.format(number))
    plt.ylabel('Number of Senator')
    plt.show()
    
draw_lobby(df_merge_lobby ,20)